In [102]:
import os
import re
import io
import sys
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
#import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from functools import reduce 
#from google.colab import files

sys.path.insert(0, "../utils")
warnings.filterwarnings('ignore')
sns.set(context='talk',style = 'whitegrid')
%matplotlib inline

In [103]:
geo2019 = pd.read_csv('geographic/ACSST5Y2019.S0701-Data.csv')
geo2019

,GEO_ID,NAME,S0701_C01_001E,S0701_C01_001M,S0701_C01_001MA,S0701_C01_001EA,S0701_C01_002E,S0701_C01_002EA,S0701_C01_002M,S0701_C01_002MA,...,S0701_C05_054MA,S0701_C05_055E,S0701_C05_055EA,S0701_C05_055M,S0701_C05_055MA,S0701_C05_056E,S0701_C05_056EA,S0701_C05_056M,S0701_C05_056MA,Unnamed: 1122
0,Geography,Geographic Area Name,Estimate!!Total!!Population 1 year and over,Margin of Error!!Total!!Population 1 year and ...,Annotation of Margin of Error!!Total!!Populati...,Annotation of Estimate!!Total!!Population 1 ye...,Estimate!!Total!!Population 1 year and over!!A...,Annotation of Estimate!!Total!!Population 1 ye...,Margin of Error!!Total!!Population 1 year and ...,Annotation of Margin of Error!!Total!!Populati...,...,Annotation of Margin of Error!!Moved; from abr...,Estimate!!Moved; from abroad!!HOUSING TENURE!!...,Annotation of Estimate!!Moved; from abroad!!HO...,Margin of Error!!Moved; from abroad!!HOUSING T...,Annotation of Margin of Error!!Moved; from abr...,Estimate!!Moved; from abroad!!PERCENT ALLOCATE...,Annotation of Estimate!!Moved; from abroad!!PE...,Margin of Error!!Moved; from abroad!!PERCENT A...,Annotation of Margin of Error!!Moved; from abr...,NaN
1,1600000US0455000,"Phoenix city, Arizona",1611604,1194,NaN,NaN,96783,NaN,1798,NaN,...,NaN,0.8,NaN,0.1,NaN,(X),(X),(X),(X),NaN
2,1600000US0644000,"Los Angeles city, California",3922917,1573,NaN,NaN,190244,NaN,2974,NaN,...,NaN,1.1,NaN,0.1,NaN,(X),(X),(X),(X),NaN
3,1600000US0664000,"Sacramento city, California",494604,638,NaN,NaN,26835,NaN,879,NaN,...,NaN,0.8,NaN,0.2,NaN,(X),(X),(X),(X),NaN
4,1600000US0667000,"San Francisco city, California",865937,666,NaN,NaN,30512,NaN,657,NaN,...,NaN,1.8,NaN,0.2,NaN,(X),(X),(X),(X),NaN
5,1600000US1354264,"Nashville city, Georgia",4785,52,NaN,NaN,354,NaN,108,NaN,...,NaN,0.7,NaN,0.9,NaN,(X),(X),(X),(X),NaN
6,1600000US1714000,"Chicago city, Illinois",2675166,1437,NaN,NaN,136955,NaN,2055,NaN,...,NaN,1.2,NaN,0.1,NaN,(X),(X),(X),(X),NaN
7,1600000US2507000,"Boston city, Massachusetts",676770,538,NaN,NaN,26876,NaN,749,NaN,...,NaN,2.2,NaN,0.2,NaN,(X),(X),(X),(X),NaN
8,1600000US3651000,"New York city, New York",8314881,2590,NaN,NaN,440536,NaN,2571,NaN,...,NaN,1.3,NaN,0.1,NaN,(X),(X),(X),(X),NaN
9,1600000US4805000,"Austin city, Texas",938887,831,NaN,NaN,49170,NaN,1249,NaN,...,NaN,1.8,NaN,0.3,NaN,(X),(X),(X),(X),NaN


In [104]:
geo2019.dtypes

GEO_ID              object
NAME                object
S0701_C01_001E      object
S0701_C01_001M      object
S0701_C01_001MA     object
                    ...   
S0701_C05_056E      object
S0701_C05_056EA     object
S0701_C05_056M      object
S0701_C05_056MA     object
Unnamed: 1122      float64
Length: 1123, dtype: object

In [105]:
#replace "(X)" in the data with NAN
geo2019 = geo2019.replace('(X)',np.NAN)
#Count all NAN values
geo2019.isna().sum().sum()

5677

In [106]:
#Delete all NaN values (this takes out all the columns with annotated values)
geo2019 = geo2019.dropna(axis='columns')
#delete GEO_ID column and row 0. They have no significance in the data.
geo2019 = geo2019.drop([0])
geo2019 = geo2019.drop(columns = ["GEO_ID"])
#rename NAME column with City
geo2019.rename(columns = {'NAME':'CITY'}, inplace = True )
geo2019

,CITY,S0701_C01_001E,S0701_C01_001M,S0701_C01_002E,S0701_C01_002M,S0701_C01_003E,S0701_C01_003M,S0701_C01_004E,S0701_C01_004M,S0701_C01_005E,...,S0701_C05_051E,S0701_C05_051M,S0701_C05_052E,S0701_C05_052M,S0701_C05_053E,S0701_C05_053M,S0701_C05_054E,S0701_C05_054M,S0701_C05_055E,S0701_C05_055M
1,"Phoenix city, Arizona",1611604,1194,96783,1798,308082,2729,159242,2761,262753,...,0.5,0.2,0.5,0.1,0.5,0.1,0.3,0.1,0.8,0.1
2,"Los Angeles city, California",3922917,1573,190244,2974,587153,3304,407505,3246,717319,...,0.6,0.1,0.6,0.1,0.8,0.1,0.4,0.1,1.1,0.1
3,"Sacramento city, California",494604,638,26835,879,82570,1555,46046,1292,92462,...,0.4,0.2,0.6,0.2,0.6,0.2,0.5,0.2,0.8,0.2
4,"San Francisco city, California",865937,666,30512,657,78010,118,62085,160,205361,...,1.5,0.5,1.2,0.1,1.3,0.1,0.7,0.1,1.8,0.2
5,"Nashville city, Georgia",4785,52,354,108,877,164,221,113,686,...,2.3,3.2,0.0,1.5,0.3,0.5,0.0,1.8,0.7,0.9
6,"Chicago city, Illinois",2675166,1437,136955,2055,395004,2410,268187,2792,538183,...,0.6,0.2,0.5,0.1,0.8,0.1,0.3,0.1,1.2,0.1
7,"Boston city, Massachusetts",676770,538,26876,749,74518,835,101601,558,166867,...,0.7,0.3,1.1,0.2,1.7,0.2,0.8,0.2,2.2,0.2
8,"New York city, New York",8314881,2590,440536,2571,1209576,291,729554,243,1501492,...,1.1,0.1,0.8,0.1,1.0,0.1,0.5,0.1,1.3,0.1
9,"Austin city, Texas",938887,831,49170,1249,132510,1389,99159,1293,214336,...,1.2,0.4,0.9,0.1,1.1,0.1,0.4,0.1,1.8,0.3
10,"Seattle city, Washington",717171,626,27295,1043,74488,1469,76705,1561,171536,...,1.9,0.7,1.2,0.2,1.5,0.2,0.5,0.1,2.5,0.3


In [107]:
#Delete margin of error columns
for col in geo2019.columns:
    if 'M' in col:
        del geo2019[col]

geo2019

,CITY,S0701_C01_001E,S0701_C01_002E,S0701_C01_003E,S0701_C01_004E,S0701_C01_005E,S0701_C01_006E,S0701_C01_007E,S0701_C01_008E,S0701_C01_009E,...,S0701_C05_046E,S0701_C05_047E,S0701_C05_048E,S0701_C05_049E,S0701_C05_050E,S0701_C05_051E,S0701_C05_052E,S0701_C05_053E,S0701_C05_054E,S0701_C05_055E
1,"Phoenix city, Arizona",1611604,96783,308082,159242,262753,225890,208337,177259,105952,...,0.2,0.3,21139,0.5,0.8,0.5,0.5,0.5,0.3,0.8
2,"Los Angeles city, California",3922917,190244,587153,407505,717319,569174,521846,438078,277893,...,0.4,0.5,17959,0.8,1.6,0.6,0.6,0.8,0.4,1.1
3,"Sacramento city, California",494604,26835,82570,46046,92462,67554,58059,55449,38967,...,0.3,0.2,18523,0.6,1.0,0.4,0.6,0.6,0.5,0.8
4,"San Francisco city, California",865937,30512,78010,62085,205361,138126,115305,101557,73974,...,1.6,1.2,52789,1.3,2.0,1.5,1.2,1.3,0.7,1.8
5,"Nashville city, Georgia",4785,354,877,221,686,572,614,555,549,...,0.0,0.0,-,0.3,0.0,2.3,0.0,0.3,0.0,0.7
6,"Chicago city, Illinois",2675166,136955,395004,268187,538183,379716,323149,297003,195049,...,0.3,0.5,17489,0.8,1.8,0.6,0.5,0.8,0.3,1.2
7,"Boston city, Massachusetts",676770,26876,74518,101601,166867,85119,74324,68604,45158,...,0.6,0.9,15536,1.7,4.5,0.7,1.1,1.7,0.8,2.2
8,"New York city, New York",8314881,440536,1209576,729554,1501492,1149333,1070062,992061,684589,...,0.5,0.7,21112,1.0,1.8,1.1,0.8,1.0,0.5,1.3
9,"Austin city, Texas",938887,49170,132510,99159,214336,152569,113381,92761,53544,...,0.5,0.6,20334,1.1,2.5,1.2,0.9,1.1,0.4,1.8
10,"Seattle city, Washington",717171,27295,74488,76705,171536,111173,88870,77164,53616,...,0.5,1.1,31073,1.5,3.3,1.9,1.2,1.5,0.5,2.5


In [108]:
city = geo2019["CITY"]
city = pd.DataFrame(data =city)
city

,CITY
1,"Phoenix city, Arizona"
2,"Los Angeles city, California"
3,"Sacramento city, California"
4,"San Francisco city, California"
5,"Nashville city, Georgia"
6,"Chicago city, Illinois"
7,"Boston city, Massachusetts"
8,"New York city, New York"
9,"Austin city, Texas"
10,"Seattle city, Washington"


In [109]:
#population for the ages between 18 t0 64
population = geo2019.iloc[:,4:9]
population.rename(columns = {'S0701_C01_004E':'Pop_18-24', 'S0701_C01_005E':'Pop_25-34','S0701_C01_006E':'Pop_35-44','S0701_C01_007E':'Pop_45-54','S0701_C01_008E':'Pop_55-64'},inplace = True)
population = population.astype(int)
population

,Pop_18-24,Pop_25-34,Pop_35-44,Pop_45-54,Pop_55-64
1,159242,262753,225890,208337,177259
2,407505,717319,569174,521846,438078
3,46046,92462,67554,58059,55449
4,62085,205361,138126,115305,101557
5,221,686,572,614,555
6,268187,538183,379716,323149,297003
7,101601,166867,85119,74324,68604
8,729554,1501492,1149333,1070062,992061
9,99159,214336,152569,113381,92761
10,76705,171536,111173,88870,77164


In [110]:
popEst = geo2019.iloc[:,170:175]
popEst.rename(columns = {'S0701_C04_004E':'Est_18-24', 'S0701_C04_005E':'Est_25-34','S0701_C04_006E':'Est_35-44','S0701_C04_007E':'Est_45-54','S0701_C04_008E':'Est_55-64'},inplace = True)
popEst = popEst.astype(float)
popEst

,Est_18-24,Est_25-34,Est_35-44,Est_45-54,Est_55-64
1,4.3,4.3,2.7,2.1,2.2
2,3.6,2.6,1.1,0.6,0.5
3,2.6,2.5,1.2,1.2,1.2
4,7.2,4.9,1.8,0.9,0.8
5,0.0,0.4,5.4,4.2,0.0
6,6.6,3.9,1.4,1.0,0.7
7,12.3,6.6,2.1,1.3,1.4
8,4.9,3.0,1.1,0.6,0.5
9,6.0,6.2,2.9,1.6,1.6
10,14.7,9.9,3.9,2.4,2.0


In [111]:
#Multiply city population by estiamted percentage of people that moved to the city from different states
population2019 = popEst.mul(population.values)
population2019

,Est_18-24,Est_25-34,Est_35-44,Est_45-54,Est_55-64
1,684740.6,1129837.9,609903.0,437507.7,389969.8
2,1467018.0,1865029.4,626091.4,313107.6,219039.0
3,119719.6,231155.0,81064.8,69670.8,66538.8
4,447012.0,1006268.9,248626.8,103774.5,81245.6
5,0.0,274.4,3088.8,2578.8,0.0
6,1770034.2,2098913.7,531602.4,323149.0,207902.1
7,1249692.3,1101322.2,178749.9,96621.2,96045.6
8,3574814.6,4504476.0,1264266.3,642037.2,496030.5
9,594954.0,1328883.2,442450.1,181409.6,148417.6
10,1127563.5,1698206.4,433574.7,213288.0,154328.0


In [112]:
#Sum each roe to give the working populatuin that moved from different states to the cities
population2019['TOTAL_19'] = population2019.sum(axis = 1)
population2019

,Est_18-24,Est_25-34,Est_35-44,Est_45-54,Est_55-64,TOTAL_19
1,684740.6,1129837.9,609903.0,437507.7,389969.8,3251959.0
2,1467018.0,1865029.4,626091.4,313107.6,219039.0,4490285.4
3,119719.6,231155.0,81064.8,69670.8,66538.8,568149.0
4,447012.0,1006268.9,248626.8,103774.5,81245.6,1886927.8
5,0.0,274.4,3088.8,2578.8,0.0,5942.0
6,1770034.2,2098913.7,531602.4,323149.0,207902.1,4931601.4
7,1249692.3,1101322.2,178749.9,96621.2,96045.6,2722431.2
8,3574814.6,4504476.0,1264266.3,642037.2,496030.5,10481624.6
9,594954.0,1328883.2,442450.1,181409.6,148417.6,2696114.5
10,1127563.5,1698206.4,433574.7,213288.0,154328.0,3626960.6


In [113]:
mobility = city
mobility['2019']= population2019['TOTAL_19']
mobility

,CITY,2019
1,"Phoenix city, Arizona",3251959.0
2,"Los Angeles city, California",4490285.4
3,"Sacramento city, California",568149.0
4,"San Francisco city, California",1886927.8
5,"Nashville city, Georgia",5942.0
6,"Chicago city, Illinois",4931601.4
7,"Boston city, Massachusetts",2722431.2
8,"New York city, New York",10481624.6
9,"Austin city, Texas",2696114.5
10,"Seattle city, Washington",3626960.6


In [114]:
geo2020 = pd.read_csv('geographic/ACSST5Y2020.S0701-Data.csv')
geo2020

,GEO_ID,NAME,S0701_C01_001E,S0701_C01_001EA,S0701_C01_001M,S0701_C01_001MA,S0701_C01_002E,S0701_C01_002EA,S0701_C01_002M,S0701_C01_002MA,...,S0701_C05_054EA,S0701_C05_055E,S0701_C05_055M,S0701_C05_055MA,S0701_C05_055EA,S0701_C05_056E,S0701_C05_056M,S0701_C05_056MA,S0701_C05_056EA,Unnamed: 1122
0,Geography,Geographic Area Name,Estimate!!Total!!Population 1 year and over,Annotation of Estimate!!Total!!Population 1 ye...,Margin of Error!!Total!!Population 1 year and ...,Annotation of Margin of Error!!Total!!Populati...,Estimate!!Total!!Population 1 year and over!!A...,Annotation of Estimate!!Total!!Population 1 ye...,Margin of Error!!Total!!Population 1 year and ...,Annotation of Margin of Error!!Total!!Populati...,...,Annotation of Estimate!!Moved; from abroad!!HO...,Estimate!!Moved; from abroad!!HOUSING TENURE!!...,Margin of Error!!Moved; from abroad!!HOUSING T...,Annotation of Margin of Error!!Moved; from abr...,Annotation of Estimate!!Moved; from abroad!!HO...,Estimate!!Moved; from abroad!!PERCENT ALLOCATE...,Margin of Error!!Moved; from abroad!!PERCENT A...,Annotation of Margin of Error!!Moved; from abr...,Annotation of Estimate!!Moved; from abroad!!PE...,NaN
1,1600000US0455000,"Phoenix city, Arizona",1637201,NaN,1140,NaN,95485,NaN,2132,NaN,...,NaN,0.8,0.1,NaN,NaN,(X),(X),(X),(X),NaN
2,1600000US0644000,"Los Angeles city, California",3933092,NaN,1557,NaN,186041,NaN,2970,NaN,...,NaN,1.0,0.1,NaN,NaN,(X),(X),(X),(X),NaN
3,1600000US0664000,"Sacramento city, California",497705,NaN,615,NaN,26719,NaN,1162,NaN,...,NaN,0.7,0.2,NaN,NaN,(X),(X),(X),(X),NaN
4,1600000US0667000,"San Francisco city, California",866648,NaN,710,NaN,31059,NaN,715,NaN,...,NaN,1.7,0.2,NaN,NaN,(X),(X),(X),(X),NaN
5,1600000US1354264,"Nashville city, Georgia",4716,NaN,87,NaN,300,NaN,121,NaN,...,NaN,0.6,1.0,NaN,NaN,(X),(X),(X),(X),NaN
6,1600000US1714000,"Chicago city, Illinois",2665195,NaN,1546,NaN,131692,NaN,2481,NaN,...,NaN,1.0,0.1,NaN,NaN,(X),(X),(X),(X),NaN
7,1600000US2507000,"Boston city, Massachusetts",681962,NaN,661,NaN,26675,NaN,666,NaN,...,NaN,2.1,0.3,NaN,NaN,(X),(X),(X),(X),NaN
8,1600000US3651000,"New York city, New York",8276623,NaN,2783,NaN,431830,NaN,2767,NaN,...,NaN,1.3,0.1,NaN,NaN,(X),(X),(X),(X),NaN
9,1600000US4805000,"Austin city, Texas",954333,NaN,747,NaN,48361,NaN,1506,NaN,...,NaN,1.8,0.2,NaN,NaN,(X),(X),(X),(X),NaN


In [115]:
#replace "(X)" in the data with NAN
geo2020 = geo2020.replace('(X)',np.NAN)
#Count all NAN values
geo2020.isna().sum().sum()

5679

In [116]:
#Delete all NaN values (this takes out all the columns with annotated values)
geo2020 = geo2020.dropna(axis='columns')
#delete GEO_ID column and row 0. They have no significance in the data.
geo2020 = geo2020.drop([0])
geo2020 = geo2020.drop(columns = ["GEO_ID"])
#rename NAME column with City
geo2020.rename(columns = {'NAME':'CITY'}, inplace = True )
geo2020

,CITY,S0701_C01_001E,S0701_C01_001M,S0701_C01_002E,S0701_C01_002M,S0701_C01_003E,S0701_C01_003M,S0701_C01_004E,S0701_C01_004M,S0701_C01_005E,...,S0701_C05_051E,S0701_C05_051M,S0701_C05_052E,S0701_C05_052M,S0701_C05_053E,S0701_C05_053M,S0701_C05_054E,S0701_C05_054M,S0701_C05_055E,S0701_C05_055M
1,"Phoenix city, Arizona",1637201,1140,95485,2132,309144,3945,162489,2971,268623,...,0.5,0.2,0.4,0.1,0.5,0.1,0.3,0.1,0.8,0.1
2,"Los Angeles city, California",3933092,1557,186041,2970,583222,4470,396251,3813,723858,...,0.7,0.2,0.5,0.1,0.7,0.1,0.4,0.1,1.0,0.1
3,"Sacramento city, California",497705,615,26719,1162,81652,1636,44635,1362,94158,...,0.4,0.3,0.5,0.1,0.6,0.1,0.4,0.2,0.7,0.2
4,"San Francisco city, California",866648,710,31059,715,78168,135,60846,218,205012,...,1.7,0.6,1.1,0.1,1.3,0.2,0.7,0.2,1.7,0.2
5,"Nashville city, Georgia",4716,87,300,121,724,187,318,158,821,...,2.5,3.8,0.0,1.6,0.4,0.5,0.0,2.4,0.6,1.0
6,"Chicago city, Illinois",2665195,1546,131692,2481,386824,4071,263662,2909,543091,...,0.5,0.1,0.5,0.1,0.7,0.1,0.3,0.1,1.0,0.1
7,"Boston city, Massachusetts",681962,661,26675,666,74553,823,101870,744,168227,...,0.8,0.4,1.2,0.2,1.7,0.2,0.9,0.2,2.1,0.3
8,"New York city, New York",8276623,2783,431830,2767,1202710,186,708657,325,1492460,...,1.0,0.2,0.8,0.1,1.0,0.1,0.4,0.1,1.3,0.1
9,"Austin city, Texas",954333,747,48361,1506,132226,2641,98399,1773,217551,...,1.2,0.5,0.9,0.1,1.1,0.1,0.4,0.1,1.8,0.2
10,"Seattle city, Washington",734270,643,27210,1155,73560,1785,76635,1814,182876,...,1.5,0.5,1.2,0.2,1.4,0.2,0.6,0.1,2.3,0.3


In [117]:
#Delete margin of error columns
for col in geo2020.columns:
    if 'M' in col:
        del geo2020[col]

geo2020

,CITY,S0701_C01_001E,S0701_C01_002E,S0701_C01_003E,S0701_C01_004E,S0701_C01_005E,S0701_C01_006E,S0701_C01_007E,S0701_C01_008E,S0701_C01_009E,...,S0701_C05_046E,S0701_C05_047E,S0701_C05_048E,S0701_C05_049E,S0701_C05_050E,S0701_C05_051E,S0701_C05_052E,S0701_C05_053E,S0701_C05_054E,S0701_C05_055E
1,"Phoenix city, Arizona",1637201,95485,309144,162489,268623,229855,209148,180455,112070,...,0.3,0.2,22249,0.5,0.7,0.5,0.4,0.5,0.3,0.8
2,"Los Angeles city, California",3933092,186041,583222,396251,723858,571032,516798,445103,290863,...,0.5,0.4,18720,0.7,1.4,0.7,0.5,0.7,0.4,1.0
3,"Sacramento city, California",497705,26719,81652,44635,94158,69148,58118,56146,40097,...,0.3,0.1,14688,0.6,1.1,0.4,0.5,0.6,0.4,0.7
4,"San Francisco city, California",866648,31059,78168,60846,205012,138196,114165,101045,76345,...,1.4,1.2,54678,1.3,2.2,1.7,1.1,1.3,0.7,1.7
5,"Nashville city, Georgia",4716,300,724,318,821,446,502,693,505,...,0.0,0.0,-,0.4,0.0,2.5,0.0,0.4,0.0,0.6
6,"Chicago city, Illinois",2665195,131692,386824,263662,543091,378306,320356,299090,198316,...,0.2,0.4,20091,0.7,1.7,0.5,0.5,0.7,0.3,1.0
7,"Boston city, Massachusetts",681962,26675,74553,101870,168227,86463,73036,69854,46846,...,0.5,0.7,22240,1.7,4.0,0.8,1.2,1.7,0.9,2.1
8,"New York city, New York",8276623,431830,1202710,708657,1492460,1142869,1051139,994445,702799,...,0.5,0.7,21741,1.0,1.7,1.0,0.8,1.0,0.4,1.3
9,"Austin city, Texas",954333,48361,132226,98399,217551,157966,114260,94537,57995,...,0.3,0.6,20461,1.1,2.5,1.2,0.9,1.1,0.4,1.8
10,"Seattle city, Washington",734270,27210,73560,76635,182876,114601,90404,76188,56399,...,1.4,1.0,37534,1.4,3.3,1.5,1.2,1.4,0.6,2.3


In [118]:
#population for the ages between 18 t0 64
population = geo2020.iloc[:,4:9]
population.rename(columns = {'S0701_C01_004E':'Pop_18-24', 'S0701_C01_005E':'Pop_25-34','S0701_C01_006E':'Pop_35-44','S0701_C01_007E':'Pop_45-54','S0701_C01_008E':'Pop_55-64'},inplace = True)
population = population.astype(int)
population

,Pop_18-24,Pop_25-34,Pop_35-44,Pop_45-54,Pop_55-64
1,162489,268623,229855,209148,180455
2,396251,723858,571032,516798,445103
3,44635,94158,69148,58118,56146
4,60846,205012,138196,114165,101045
5,318,821,446,502,693
6,263662,543091,378306,320356,299090
7,101870,168227,86463,73036,69854
8,708657,1492460,1142869,1051139,994445
9,98399,217551,157966,114260,94537
10,76635,182876,114601,90404,76188


In [119]:
popEst = geo2020.iloc[:,170:175]
popEst.rename(columns = {'S0701_C04_004E':'Est_18-24', 'S0701_C04_005E':'Est_25-34','S0701_C04_006E':'Est_35-44','S0701_C04_007E':'Est_45-54','S0701_C04_008E':'Est_55-64'},inplace = True)
popEst = popEst.astype(float)
popEst

,Est_18-24,Est_25-34,Est_35-44,Est_45-54,Est_55-64
1,4.6,4.1,2.7,2.0,2.3
2,3.5,2.5,1.1,0.6,0.4
3,2.7,2.4,1.1,1.1,1.2
4,6.4,4.6,2.0,0.8,0.7
5,0.0,0.4,4.0,2.6,0.0
6,7.0,3.9,1.6,1.0,0.8
7,12.7,6.5,2.4,1.3,1.6
8,5.0,3.1,1.0,0.6,0.5
9,6.4,6.3,2.6,1.9,1.4
10,15.0,10.6,3.6,2.4,1.8


In [120]:
#Multiply city population by estiamted percentage of people that moved to the city from different states
population2020 = popEst.mul(population.values)
population2020

,Est_18-24,Est_25-34,Est_35-44,Est_45-54,Est_55-64
1,747449.4,1101354.3,620608.5,418296.0,415046.5
2,1386878.5,1809645.0,628135.2,310078.8,178041.2
3,120514.5,225979.2,76062.8,63929.8,67375.2
4,389414.4,943055.2,276392.0,91332.0,70731.5
5,0.0,328.4,1784.0,1305.2,0.0
6,1845634.0,2118054.9,605289.6,320356.0,239272.0
7,1293749.0,1093475.5,207511.2,94946.8,111766.4
8,3543285.0,4626626.0,1142869.0,630683.4,497222.5
9,629753.6,1370571.3,410711.6,217094.0,132351.8
10,1149525.0,1938485.6,412563.6,216969.6,137138.4


In [121]:
#Sum each roe to give the working populatuin that moved from different states to the cities
population2020['TOTAL_20'] = population2020.sum(axis = 1)
population2020

,Est_18-24,Est_25-34,Est_35-44,Est_45-54,Est_55-64,TOTAL_20
1,747449.4,1101354.3,620608.5,418296.0,415046.5,3302754.7
2,1386878.5,1809645.0,628135.2,310078.8,178041.2,4312778.7
3,120514.5,225979.2,76062.8,63929.8,67375.2,553861.5
4,389414.4,943055.2,276392.0,91332.0,70731.5,1770925.1
5,0.0,328.4,1784.0,1305.2,0.0,3417.6
6,1845634.0,2118054.9,605289.6,320356.0,239272.0,5128606.5
7,1293749.0,1093475.5,207511.2,94946.8,111766.4,2801448.9
8,3543285.0,4626626.0,1142869.0,630683.4,497222.5,10440685.9
9,629753.6,1370571.3,410711.6,217094.0,132351.8,2760482.3
10,1149525.0,1938485.6,412563.6,216969.6,137138.4,3854682.2


In [122]:
mobility['2020']= population2020['TOTAL_20']
mobility

,CITY,2019,2020
1,"Phoenix city, Arizona",3251959.0,3302754.7
2,"Los Angeles city, California",4490285.4,4312778.7
3,"Sacramento city, California",568149.0,553861.5
4,"San Francisco city, California",1886927.8,1770925.1
5,"Nashville city, Georgia",5942.0,3417.6
6,"Chicago city, Illinois",4931601.4,5128606.5
7,"Boston city, Massachusetts",2722431.2,2801448.9
8,"New York city, New York",10481624.6,10440685.9
9,"Austin city, Texas",2696114.5,2760482.3
10,"Seattle city, Washington",3626960.6,3854682.2


In [123]:
geo2021 = pd.read_csv('geographic/ACSST5Y2021.S0701-Data.csv')
geo2021

,GEO_ID,NAME,S0701_C01_001E,S0701_C01_001M,S0701_C01_001MA,S0701_C01_001EA,S0701_C01_002E,S0701_C01_002M,S0701_C01_002MA,S0701_C01_002EA,...,S0701_C05_054MA,S0701_C05_055E,S0701_C05_055EA,S0701_C05_055M,S0701_C05_055MA,S0701_C05_056E,S0701_C05_056EA,S0701_C05_056M,S0701_C05_056MA,Unnamed: 1122
0,Geography,Geographic Area Name,Estimate!!Total!!Population 1 year and over,Margin of Error!!Total!!Population 1 year and ...,Annotation of Margin of Error!!Total!!Populati...,Annotation of Estimate!!Total!!Population 1 ye...,Estimate!!Total!!Population 1 year and over!!A...,Margin of Error!!Total!!Population 1 year and ...,Annotation of Margin of Error!!Total!!Populati...,Annotation of Estimate!!Total!!Population 1 ye...,...,Annotation of Margin of Error!!Moved; from abr...,Estimate!!Moved; from abroad!!HOUSING TENURE!!...,Annotation of Estimate!!Moved; from abroad!!HO...,Margin of Error!!Moved; from abroad!!HOUSING T...,Annotation of Margin of Error!!Moved; from abr...,Estimate!!Moved; from abroad!!PERCENT ALLOCATE...,Annotation of Estimate!!Moved; from abroad!!PE...,Margin of Error!!Moved; from abroad!!PERCENT A...,Annotation of Margin of Error!!Moved; from abr...,NaN
1,1600000US0455000,"Phoenix city, Arizona",1572364,1042,NaN,NaN,88672,2372,NaN,NaN,...,NaN,0.7,NaN,0.2,NaN,(X),(X),(X),(X),NaN
2,1600000US0644000,"Los Angeles city, California",3862877,1469,NaN,NaN,175644,3189,NaN,NaN,...,NaN,0.9,NaN,0.1,NaN,(X),(X),(X),(X),NaN
3,1600000US0664000,"Sacramento city, California",512941,516,NaN,NaN,27062,1193,NaN,NaN,...,NaN,0.6,NaN,0.1,NaN,(X),(X),(X),(X),NaN
4,1600000US0667000,"San Francisco city, California",857787,744,NaN,NaN,30073,767,NaN,NaN,...,NaN,1.7,NaN,0.2,NaN,(X),(X),(X),(X),NaN
5,1600000US1354264,"Nashville city, Georgia",4790,69,NaN,NaN,363,156,NaN,NaN,...,NaN,0.0,NaN,2.0,NaN,(X),(X),(X),(X),NaN
6,1600000US1714000,"Chicago city, Illinois",2709340,1535,NaN,NaN,130344,2158,NaN,NaN,...,NaN,1.0,NaN,0.1,NaN,(X),(X),(X),(X),NaN
7,1600000US2507000,"Boston city, Massachusetts",666462,575,NaN,NaN,25638,824,NaN,NaN,...,NaN,2.0,NaN,0.2,NaN,(X),(X),(X),(X),NaN
8,1600000US3651000,"New York city, New York",8631602,2884,NaN,NaN,438992,2891,NaN,NaN,...,NaN,1.1,NaN,0.1,NaN,(X),(X),(X),(X),NaN
9,1600000US4805000,"Austin city, Texas",934183,914,NaN,NaN,44636,1415,NaN,NaN,...,NaN,2.0,NaN,0.4,NaN,(X),(X),(X),(X),NaN


In [124]:
geo2021.dtypes

GEO_ID              object
NAME                object
S0701_C01_001E      object
S0701_C01_001M      object
S0701_C01_001MA     object
                    ...   
S0701_C05_056E      object
S0701_C05_056EA     object
S0701_C05_056M      object
S0701_C05_056MA     object
Unnamed: 1122      float64
Length: 1123, dtype: object

In [125]:
#replace "(X)" in the data with NAN
geo2021 = geo2021.replace('(X)',np.NAN)
#Count all NAN values
geo2021.isna().sum().sum()

5679

In [126]:
#Delete all NaN values (this takes out all the columns with annotated values)
geo2021 = geo2021.dropna(axis='columns')
#delete GEO_ID column and row 0. They have no significance in the data.
geo2021 = geo2021.drop([0])
geo2021 = geo2021.drop(columns = ["GEO_ID"])
#rename NAME column with City
geo2021.rename(columns = {'NAME':'CITY'}, inplace = True )
geo2021

,CITY,S0701_C01_001E,S0701_C01_001M,S0701_C01_002E,S0701_C01_002M,S0701_C01_003E,S0701_C01_003M,S0701_C01_004E,S0701_C01_004M,S0701_C01_005E,...,S0701_C05_051E,S0701_C05_051M,S0701_C05_052E,S0701_C05_052M,S0701_C05_053E,S0701_C05_053M,S0701_C05_054E,S0701_C05_054M,S0701_C05_055E,S0701_C05_055M
1,"Phoenix city, Arizona",1572364,1042,88672,2372,295674,3648,155760,3316,257908,...,0.4,0.2,0.4,0.1,0.4,0.1,0.3,0.1,0.7,0.2
2,"Los Angeles city, California",3862877,1469,175644,3189,577644,4817,381670,3824,699729,...,0.7,0.1,0.5,0.1,0.7,0.1,0.3,0.1,0.9,0.1
3,"Sacramento city, California",512941,516,27062,1193,83774,2019,46187,1670,94989,...,0.5,0.4,0.5,0.1,0.5,0.1,0.5,0.2,0.6,0.1
4,"San Francisco city, California",857787,744,30073,767,79975,191,58770,180,195662,...,1.7,0.7,1.1,0.1,1.2,0.2,0.6,0.2,1.7,0.2
5,"Nashville city, Georgia",4790,69,363,156,637,202,245,121,744,...,0.0,4.7,0.0,1.7,0.0,1.0,0.0,2.1,0.0,2.0
6,"Chicago city, Illinois",2709340,1535,130344,2158,395908,3621,263317,3105,543502,...,0.6,0.2,0.5,0.1,0.7,0.1,0.3,0.1,1.0,0.1
7,"Boston city, Massachusetts",666462,575,25638,824,73860,1057,100005,891,161128,...,0.7,0.3,1.2,0.2,1.5,0.2,0.7,0.2,2.0,0.2
8,"New York city, New York",8631602,2884,438992,2891,1278944,263,721971,470,1515476,...,0.8,0.1,0.7,0.1,0.9,0.1,0.4,0.1,1.1,0.1
9,"Austin city, Texas",934183,914,44636,1415,128726,2812,97464,1648,211863,...,1.2,0.6,0.9,0.1,1.2,0.2,0.3,0.1,2.0,0.4
10,"Seattle city, Washington",719371,636,26909,1169,71743,1863,73762,1820,180082,...,1.7,0.8,1.2,0.2,1.4,0.2,0.6,0.1,2.2,0.3


In [127]:
#Delete margin of error columns
for col in geo2021.columns:
    if 'M' in col:
        del geo2021[col]

geo2021

,CITY,S0701_C01_001E,S0701_C01_002E,S0701_C01_003E,S0701_C01_004E,S0701_C01_005E,S0701_C01_006E,S0701_C01_007E,S0701_C01_008E,S0701_C01_009E,...,S0701_C05_046E,S0701_C05_047E,S0701_C05_048E,S0701_C05_049E,S0701_C05_050E,S0701_C05_051E,S0701_C05_052E,S0701_C05_053E,S0701_C05_054E,S0701_C05_055E
1,"Phoenix city, Arizona",1572364,88672,295674,155760,257908,220560,200526,177437,110600,...,0.2,0.3,21963,0.4,0.8,0.4,0.4,0.4,0.3,0.7
2,"Los Angeles city, California",3862877,175644,577644,381670,699729,568491,510210,447059,295708,...,0.3,0.4,20659,0.7,1.3,0.7,0.5,0.7,0.3,0.9
3,"Sacramento city, California",512941,27062,83774,46187,94989,73093,61076,57796,41722,...,0.0,0.2,14555,0.5,1.0,0.5,0.5,0.5,0.5,0.6
4,"San Francisco city, California",857787,30073,79975,58770,195662,139472,114074,101808,78589,...,1.3,1.2,63023,1.2,1.8,1.7,1.1,1.2,0.6,1.7
5,"Nashville city, Georgia",4790,363,637,245,744,435,667,758,557,...,0.0,0.0,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"Chicago city, Illinois",2709340,130344,395908,263317,543502,391778,325933,308115,208056,...,0.5,0.4,18540,0.7,1.6,0.6,0.5,0.7,0.3,1.0
7,"Boston city, Massachusetts",666462,25638,73860,100005,161128,86159,71064,69149,47358,...,1.0,0.7,28813,1.5,3.5,0.7,1.2,1.5,0.7,2.0
8,"New York city, New York",8631602,438992,1278944,721971,1515476,1206501,1095182,1056332,757463,...,0.5,0.7,24783,0.9,1.5,0.8,0.7,0.9,0.4,1.1
9,"Austin city, Texas",934183,44636,128726,97464,211863,158617,112328,92346,57069,...,0.5,0.6,21597,1.2,3.0,1.2,0.9,1.2,0.3,2.0
10,"Seattle city, Washington",719371,26909,71743,73762,180082,114368,89215,73829,55845,...,1.3,1.0,42317,1.4,2.6,1.7,1.2,1.4,0.6,2.2


In [128]:
#population for the ages between 18 t0 64
population = geo2021.iloc[:,4:9]
population.rename(columns = {'S0701_C01_004E':'Pop_18-24', 'S0701_C01_005E':'Pop_25-34','S0701_C01_006E':'Pop_35-44','S0701_C01_007E':'Pop_45-54','S0701_C01_008E':'Pop_55-64'},inplace = True)
population = population.astype(int)
population

,Pop_18-24,Pop_25-34,Pop_35-44,Pop_45-54,Pop_55-64
1,155760,257908,220560,200526,177437
2,381670,699729,568491,510210,447059
3,46187,94989,73093,61076,57796
4,58770,195662,139472,114074,101808
5,245,744,435,667,758
6,263317,543502,391778,325933,308115
7,100005,161128,86159,71064,69149
8,721971,1515476,1206501,1095182,1056332
9,97464,211863,158617,112328,92346
10,73762,180082,114368,89215,73829


In [129]:
popEst = geo2021.iloc[:,170:175]
popEst.rename(columns = {'S0701_C04_004E':'Est_18-24', 'S0701_C04_005E':'Est_25-34','S0701_C04_006E':'Est_35-44','S0701_C04_007E':'Est_45-54','S0701_C04_008E':'Est_55-64'},inplace = True)
popEst = popEst.astype(float)
popEst

,Est_18-24,Est_25-34,Est_35-44,Est_45-54,Est_55-64
1,4.9,4.1,2.5,2.0,2.5
2,3.6,2.5,1.1,0.6,0.4
3,2.5,1.9,1.2,0.8,1.0
4,6.4,4.5,1.9,0.8,0.6
5,0.0,0.0,4.6,1.9,0.0
6,7.0,3.9,1.5,1.0,0.8
7,13.6,6.5,2.5,1.5,1.4
8,5.2,3.1,1.0,0.6,0.5
9,6.8,6.5,2.9,2.1,1.4
10,14.5,10.6,3.5,2.4,1.5


In [130]:
#Multiply city population by estiamted percentage of people that moved to the city from different states
population2021 = popEst.mul(population.values)
population2021

,Est_18-24,Est_25-34,Est_35-44,Est_45-54,Est_55-64
1,763224.0,1057422.8,551400.0,401052.0,443592.5
2,1374012.0,1749322.5,625340.1,306126.0,178823.6
3,115467.5,180479.1,87711.6,48860.8,57796.0
4,376128.0,880479.0,264996.8,91259.2,61084.8
5,0.0,0.0,2001.0,1267.3,0.0
6,1843219.0,2119657.8,587667.0,325933.0,246492.0
7,1360068.0,1047332.0,215397.5,106596.0,96808.6
8,3754249.2,4697975.6,1206501.0,657109.2,528166.0
9,662755.2,1377109.5,459989.3,235888.8,129284.4
10,1069549.0,1908869.2,400288.0,214116.0,110743.5


In [131]:
#Sum each roe to give the working populatuin that moved from different states to the cities
population2021['TOTAL_21'] = population2021.sum(axis = 1)
population2021

,Est_18-24,Est_25-34,Est_35-44,Est_45-54,Est_55-64,TOTAL_21
1,763224.0,1057422.8,551400.0,401052.0,443592.5,3216691.3
2,1374012.0,1749322.5,625340.1,306126.0,178823.6,4233624.2
3,115467.5,180479.1,87711.6,48860.8,57796.0,490315.0
4,376128.0,880479.0,264996.8,91259.2,61084.8,1673947.8
5,0.0,0.0,2001.0,1267.3,0.0,3268.3
6,1843219.0,2119657.8,587667.0,325933.0,246492.0,5122968.8
7,1360068.0,1047332.0,215397.5,106596.0,96808.6,2826202.1
8,3754249.2,4697975.6,1206501.0,657109.2,528166.0,10844001.0
9,662755.2,1377109.5,459989.3,235888.8,129284.4,2865027.2
10,1069549.0,1908869.2,400288.0,214116.0,110743.5,3703565.7


In [132]:
mobility['2021']= population2021['TOTAL_21']
mobility

,CITY,2019,2020,2021
1,"Phoenix city, Arizona",3251959.0,3302754.7,3216691.3
2,"Los Angeles city, California",4490285.4,4312778.7,4233624.2
3,"Sacramento city, California",568149.0,553861.5,490315.0
4,"San Francisco city, California",1886927.8,1770925.1,1673947.8
5,"Nashville city, Georgia",5942.0,3417.6,3268.3
6,"Chicago city, Illinois",4931601.4,5128606.5,5122968.8
7,"Boston city, Massachusetts",2722431.2,2801448.9,2826202.1
8,"New York city, New York",10481624.6,10440685.9,10844001.0
9,"Austin city, Texas",2696114.5,2760482.3,2865027.2
10,"Seattle city, Washington",3626960.6,3854682.2,3703565.7


In [133]:
mobility.set_index('CITY')

,2019,2020,2021
CITY,,,
"Phoenix city, Arizona",3251959.0,3302754.7,3216691.3
"Los Angeles city, California",4490285.4,4312778.7,4233624.2
"Sacramento city, California",568149.0,553861.5,490315.0
"San Francisco city, California",1886927.8,1770925.1,1673947.8
"Nashville city, Georgia",5942.0,3417.6,3268.3
"Chicago city, Illinois",4931601.4,5128606.5,5122968.8
"Boston city, Massachusetts",2722431.2,2801448.9,2826202.1
"New York city, New York",10481624.6,10440685.9,10844001.0
"Austin city, Texas",2696114.5,2760482.3,2865027.2


In [134]:
mobility.describe()

,2019,2020,2021
count,1.000000e+01,1.000000e+01,1.000000e+01
mean,3.466200e+06,3.492964e+06,3.497961e+06
std,2.915409e+06,2.917013e+06,3.031414e+06
min,5.942000e+03,3.417600e+03,3.268300e+03
25%,2.089224e+06,2.018314e+06,1.962011e+06
50%,2.987195e+06,3.052102e+06,3.040859e+06
75%,4.274454e+06,4.198255e+06,4.101110e+06
max,1.048162e+07,1.044069e+07,1.084400e+07


In [136]:
mobility.to_csv('mobility_df.csv',index = False)
mobility

,CITY,2019,2020,2021
1,"Phoenix city, Arizona",3251959.0,3302754.7,3216691.3
2,"Los Angeles city, California",4490285.4,4312778.7,4233624.2
3,"Sacramento city, California",568149.0,553861.5,490315.0
4,"San Francisco city, California",1886927.8,1770925.1,1673947.8
5,"Nashville city, Georgia",5942.0,3417.6,3268.3
6,"Chicago city, Illinois",4931601.4,5128606.5,5122968.8
7,"Boston city, Massachusetts",2722431.2,2801448.9,2826202.1
8,"New York city, New York",10481624.6,10440685.9,10844001.0
9,"Austin city, Texas",2696114.5,2760482.3,2865027.2
10,"Seattle city, Washington",3626960.6,3854682.2,3703565.7
